In [1]:
import pandas as pd
import baostock as bs
import function_book as f
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
import cmath 
import csv

In [2]:
fundid = 'fund000404'
report_date_list = ['2019-09-30','2019-12-31','2020-03-31','2020-06-30','2020-08-24']
net_value_fund_filename = "net_value_"+str(fundid)+"_"+report_date_list[0]+".csv"
fund_value = pd.read_csv(net_value_fund_filename)
fund_value.set_index(keys = 'date',inplace=True)
hs300 = pd.read_csv("hs300_2018-01-01.csv")
hs300.set_index(keys = 'date',inplace=True)

In [4]:
fund_value.head(3)

,unit_net_value,cum_net_value,daily_return,subscription_status,redemption_status,dividend
date,,,,,,
2019-09-30,2.408,2.408,-1.27,True,True,NaN
2019-10-08,2.357,2.357,-2.12,True,True,NaN
2019-10-09,2.381,2.381,1.02,True,True,NaN


In [5]:
hs300.head(3)

,sh.000300
date,
2018-01-02,4087.401
2018-01-03,4111.392
2018-01-04,4128.811


In [ ]:
# generate file for regression
#("reg_"+filename+'.csv',index = True ,index_label = 'date')
for i in range(1,len(report_date_list)-1):
    # i = 1,2,3
    print(report_date_list[i])
    f.get_table_for_reg(fundid,report_date_list,i,fund_value,hs300)

In [ ]:
# create a file for saving result
with open('result.csv', "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['fundid','date',    'reg',         'train_r2','test_r2','train_var','test_var','test_var1','test_var2','ratio','disc','comments','coef','intercept'])
    f.close()

In [ ]:
for i in range(1,len(report_date_list)-1):
    fund_value = pd.read_csv("reg_"+fundid+"_"+report_date_list[i]+".csv")
    fund_value.set_index(keys = 'date',inplace=True)
    report_date = report_date_list[i]
    start_date =report_date_list[i-1]
    end_date = report_date_list[i+1]

    stocks_details= pd.read_csv(fundid+"_"+report_date_list[i]+".csv")
    report_date_fund_value = stocks_details['fund_value'][0]/(stocks_details['ratio'][0]/100)
    ratio = stocks_details['fund_value'].sum()/report_date_fund_value

    if 'dist' in fund_value.columns:
        pass
    else:
        fund_value['dist'] = np.arange(0,len(fund_value),1)
        temp = fund_value.loc[report_date ,'dist']
        pos = len(fund_value.loc[:report_date])
        fund_value['dist'][pos:] =fund_value['dist'][pos:].apply(lambda x:x-temp)
        fund_value['dist'][:pos ] =fund_value['dist'][:pos ].apply(lambda x:temp-x)
        print("save")
        fund_value.to_csv("reg_"+fundid+"_"+report_date_list[i]+".csv",index = True ,index_label = 'date')

    mydf =fund_value[['daily_return','stocks_daily_return','hs300_daily_return','dist']].copy()
    mydf.dropna(inplace = True)

    x_list = ['stocks_daily_return','hs300_daily_return']
    comments = 'n'
    disc1 = 1
    ratio1 = 1
    f.reg_sk(mydf.copy(),x_list,comments,disc1,fundid,report_date,ratio1)

    x_list = ['c_stocks_daily_return','c_hs300_daily_return']
    comments = 'c'
    for disc in [0.98,0.985,0.99,0.995]:
        print(report_date,disc)
        f.reg_sk(mydf.copy(),x_list,comments,disc,fundid,report_date,ratio)